In [2]:
sc

In [49]:
lines = sc.textFile("input/file1")
lines

input/file1 MapPartitionsRDD[16] at textFile at NativeMethodAccessorImpl.java:-2

In [50]:
lines.collect()

[u'Hello World Bye World']

In [51]:
words = lines.flatMap(lambda l: l.split(" "))

In [52]:
words.count()

4

## Problem 1

In [60]:
files = sc.wholeTextFiles("input", use_unicode=False)

In [61]:
files.collect()

[('file:/home/vagrant/csds-material/input/file2',
  'Hello Hadoop Goodbye Hadoop\n'),
 ('file:/home/vagrant/csds-material/input/file1', 'Hello World Bye World\n')]

In [70]:
words = files.values().flatMap(lambda l: l.strip('\n').split(" "))

In [106]:
rdd = words.map(lambda p: (p, 1))

In [107]:
sorted(rdd.groupByKey().mapValues(len).collect())

[('Bye', 1), ('Goodbye', 1), ('Hadoop', 2), ('Hello', 2), ('World', 2)]

## Problem 2

In [289]:
from pyspark.sql.types import *
import pyspark.sql

In [290]:
file2 = sc.textFile("hive/purchases.txt", use_unicode=False)

In [291]:
parts = file2.map(lambda l: l.split(","))
purchases = parts.map(lambda p: (p[0], p[1], p[2], p[3], p[4]))
schemaString = "timestamp location category price card"
fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)
sqlContext = SQLContext(sc)
schemapurchases =  sqlContext.createDataFrame(purchases,schema)
schemapurchases.registerTempTable("purchases")

###### Q1: What is the average price of the products that were purchased via Mastercard?

In [317]:
sqlContext.sql("SELECT avg(price) FROM purchases where card = 'MasterCard'").show()

c0                
275.06773195876286


###### Q2: Which date recorded the highest total sales?

In [316]:
sqlContext.sql("SELECT CAST(CAST(timestamp AS TIMESTAMP) AS DATE), SUM(price) \
               FROM purchases group by CAST(CAST(timestamp AS TIMESTAMP) AS DATE) order by c1 desc limit 1").show()

c0         c1                
2012-03-17 2384.4800000000005


    + Q3: What is the minimum value of a product under the Computers category?

In [323]:
sqlContext.sql("SELECT price FROM purchases where category = 'Computers' order by price limit 1").show()

price
0.38 


    + Q4: How many distinct categories of products are there?

In [332]:
sqlContext.sql("SELECT COUNT(Distinct category) FROM purchases").show()

c0
18


    + Q5: Which store location had the lowest total sales?

In [331]:
sqlContext.sql("SELECT SUM(price) FROM purchases group by location order by c0 limit 1").show()

c0    
784.96
